In [16]:
import pandas as pd
import sys, getopt
import numpy as np
import pprint
import os
from datetime import datetime
import glob
import pandas as pd
import emgimporter
import pickle
import time
import importlib


from keras.layers.convolutional import Conv1D
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LocallyConnected2D
from keras.layers import Conv2D, MaxPooling2D, LSTM, Conv1D, MaxPooling1D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.models import load_model


In [138]:
class Classifier:
    def __init__(self,modelPath=None):
        self.loadModel(modelPath)
        pass
            
    def loadModel(self,modelPath):
        try:
            self.model = load_model(modelPath)
            print("Model " + str(modelPath) + " loaded sucessfully")
            return True
        except:
            print("Model " + str(modelPath) + " failure!")
            return False
            
    def printModelSummary(self):
        try:
            print("----------------------------\nModel Summary:")
            print(self.model.summary())
            print("----------------------------\n")
        except:
            print("----------------------------\n")
            print("Model does not exist")
            print("----------------------------\n")
            
    def classify(self,emg_sample):
        return np.squeeze(self.model.predict(np.expand_dims(np.expand_dims(np.array(emg_sample,ndmin=2),axis=1),axis=-1)))
    
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [94]:
GPU = False
modelPath = "/home/myo/models/network_1_100_50_10000.h5"

In [95]:
# Process Variables & Load all stuff
if not GPU:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
if not modelPath == None:
    cf = Classifier(modelPath)
    cf.printModelSummary()


Model /home/myo/models/network_1_100_50_10000.h5 loaded sucessfully
----------------------------
Model Summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 1, 1, 50)          450       
_________________________________________________________________
activation_1 (Activation)    (None, 1, 1, 50)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1, 50)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               5100      
_________________________________________________________________
activation_2 (Activation)    (None, 100)               0         
______________________________

In [96]:
dic = { 1: 'Tasse aufnehmen',
        2: 'Tasse halten',
        3: 'Tasse abstellen',
        4: 'Tasse hoch&runter',
        8: 'Ruhe (Supination)',
        9: 'Ruhe (Pronation)'
      }
its = ["Sensorhandschuh_fix_reib",
       "Sensorhandschuh",
       "Sensorhandschuh_fix_einarm_wein",
       "Sensorhandschuh_fix_henkel",
       "Sensorhandschuh_fix"
       
      ]
path = ['emg_data','ipa_emg','SensorHandSchuh']


In [122]:
feed_dic = emgimporter.import_folder(its,dic,path)
feed_dic['Sensorhandschuh']

[(1, [      0   1    2    3    4    5    6    7
   0    94  58  127  107   72  301  151   78
   1    99  52  122   92   63  344  149   79
   2    68  51   88   82   60  368  127   78
   3    75  51   94   72   54  369  205   87
   4   128  54   88   73   57  374  210  124
   5   191  46   79   81   60  411  206  138
   6   199  44   66   92   63  387  211  141
   7   209  50   83   98   60  310  241  144
   8   211  55  105  135   68  338  199  148
   9   190  67  119  140   63  393  215  130
   10  130  70  147  153   71  390  217  117
   11  144  73  147  151   73  370  215  129
   12  163  74  143  148   74  373  204  126
   13  198  73  135  137   79  393  228  134
   14  215  75  161  135   82  334  219  127
   15  219  73  144  126   75  269  222  124
   16  211  74  178  125   74  270  219  107
   17  199  89  215  143   81  269  205  111
   18  168  90  210  128   75  194  164   85
   19  163  82  183  139   98  202  197   92
   20  164  80  178  139  109  208  209  106
   21  

In [98]:
feed_dic

{'Sensorhandschuh': [(1, [      0   1    2    3    4    5    6    7
    0    94  58  127  107   72  301  151   78
    1    99  52  122   92   63  344  149   79
    2    68  51   88   82   60  368  127   78
    3    75  51   94   72   54  369  205   87
    4   128  54   88   73   57  374  210  124
    5   191  46   79   81   60  411  206  138
    6   199  44   66   92   63  387  211  141
    7   209  50   83   98   60  310  241  144
    8   211  55  105  135   68  338  199  148
    9   190  67  119  140   63  393  215  130
    10  130  70  147  153   71  390  217  117
    11  144  73  147  151   73  370  215  129
    12  163  74  143  148   74  373  204  126
    13  198  73  135  137   79  393  228  134
    14  215  75  161  135   82  334  219  127
    15  219  73  144  126   75  269  222  124
    16  211  74  178  125   74  270  219  107
    17  199  89  215  143   81  269  205  111
    18  168  90  210  128   75  194  164   85
    19  163  82  183  139   98  202  197   92
    20  164 

In [192]:
out_dic = dict()
for i in feed_dic:
    print("Dict key: " + str(i))
    dictlist = list()
    for element in feed_dic[i]:
        outTuple = tuple()
        tmpList =list()
        for element2 in element[1]:
            singleRow = list()
            for row in element2.values:
                tmp = cf.classify(row)
                singleRow.append(tmp[1]/(tmp[0] + tmp[1]))
            #print(singleRow)
            if len(singleRow) > 0:
                tmpList.append(singleRow)
        if len(tmpList) > 0:
            outTuple = (element[0],tmpList)
            dictlist.append(outTuple)
    if len(dictlist) > 0:  
        out_dic[i] = dictlist
        

Dict key: Sensorhandschuh_fix_reib
Dict key: Sensorhandschuh
Dict key: Sensorhandschuh_fix_einarm_wein
Dict key: Sensorhandschuh_fix_henkel
Dict key: Sensorhandschuh_fix


In [144]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.layouts import column, row
from bokeh.palettes import Spectral11, Category20, Set1
from bokeh.models import Range1d
output_notebook()

Loading BokehJS ...

In [164]:
def plotClassifier(name="Sensorhandschuh",cMatrix=[0,0], bufferSize = 50):
    plotable = out_dic[name][cMatrix[0]][1][cMatrix[1]]
    p = figure()
    y = moving_average(plotable,bufferSize)
    p.line(x=range(0,len(y)),y=y)
    show(p)

In [165]:
l = list([1,2,3,4])
it = iter(l)

In [171]:
i = 1
keys = list(out_dic.keys())

In [186]:
name = keys[i%len(keys)]
bufferSize = 50
plotable = out_dic[name][0][1][cMatrix[1]]
p = figure()
y = moving_average(plotable,bufferSize)
p.line(x=range(0,len(y)),y=y)
show(p)
i += 1

IndexError: list index out of range

In [190]:
out_dic[keys[0]]

[(1, []),
 (2,
  [[2.3764067e-05,
    3.8279113e-06,
    5.9220818e-12,
    4.3390134e-31,
    0.0,
    1.2198016e-38,
    4.698814e-22,
    0.31971636,
    0.15550882,
    8.1928459e-20,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    2.8596547e-38,
    0.0,
    1.810954e-36,
    3.1921624e-10,
    3.6609017e-05,
    0.0049162921,
    0.99552327,
    1.1076137e-06,
    0.0025251303,
    0.10158534,
    2.9522005e-21,
    2.5249461e-23,
    0.98300767,
    0.99999952,
    0.45038444,
    4.3793883e-05,
    0.046666957,
    0.99790043,
    1.4269241e-32,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    3.689315e-34,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    7.8661172e-38,
    0.0,
    0.0,
    0.0,
    0.0,
    0.